## Import libraries

In [116]:
import fitz
import json
from re import findall
from re import sub
from os import listdir
from os.path import isfile, join
import PyPDF2
import glob
import en_core_web_sm
import numpy as np  
import pandas as pd
import re  
import nltk  
from sklearn.datasets import load_files 
nltk.download('stopwords')  
import pickle  
from nltk.corpus import stopwords
import string


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdullah.kurkcu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [145]:
# All the pdf files in the folder
pdf_files = glob.glob("%s/*.pdf" % "../resumesnot/")


# Resumes -> convert to a dictionary with an employee name
resumesTest = pd.DataFrame()

texts = []
# File loop
for file in pdf_files:
    print(file)
    text = ""
    try:
        fileReader = PyPDF2.PdfFileReader(open(file,'rb'))
        count = fileReader.numPages
        text = ""
        i = 0
        while i < count:
            text += fileReader.getPage(i).extractText()
            i = i +1  
        # lowercase
        text = text.lower()
        # remove nubmers
        text = re.sub(r'\d+', '', text)
        # remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        # remove \n
        text = text.replace('\n',' ')
        # remove ndł
        text= text.replace('ndł',' ')
        # remove \nł
        text = text.replace('\nł',' ')
        # remove ł
        text = text.replace('ł',' ')
        # remove \nł
        text = text.replace('\nł',' ')

        texts.append(text)
    except Exception as e:
        print(e)
        pass
    
resumesTest['text'] = texts
resumesTest['classification'] = "NotResume"

../resumesnot\19.00150_Town of Vail_Gore Valley Trail_Feb19_FINAL_LR.pdf
../resumesnot\21.00876 Becker County Heartland Trail Engineering & Design Services - Final.pdf
../resumesnot\21.00980 Sioux City Proposal.pdf


In [151]:
# All the pdf files in the folder
pdf_files = glob.glob("%s/*.pdf" % "../resumes/")


# Resumes -> convert to a dictionary with an employee name
resumes = pd.DataFrame()

texts = []
# File loop
for file in pdf_files:
    print(file)
    text = ""
    try:
        fileReader = PyPDF2.PdfFileReader(open(file,'rb'))
        count = fileReader.numPages
        text = ""
        i = 0
        while i < count:
            text += fileReader.getPage(i).extractText()
            i = i +1  
        # lowercase
        text = text.lower()
        # remove nubmers
        text = re.sub(r'\d+', '', text)
        # remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        # remove \n
        text = text.replace('\n',' ')
        # remove ndł
        text= text.replace('ndł',' ')
        # remove \nł
        text = text.replace('\nł',' ')
        # remove ł
        text = text.replace('ł',' ')
        # remove \nł
        text = text.replace('\nł',' ')

        texts.append(text)
    except Exception as e:
        print(e)
        pass
    
resumes['text'] = texts
resumes['classification'] = "Resume"

../resumes\17.00532 0.pdf
../resumes\17.00532 1.pdf
../resumes\17.00532 2.pdf
../resumes\17.00532 3.pdf
../resumes\18.00308 0.pdf
../resumes\18.00308 1.pdf
../resumes\18.00308 2.pdf
../resumes\18.00308 3.pdf
../resumes\19.00064 0.pdf
../resumes\19.00064 1.pdf
../resumes\19.00064 10.pdf
../resumes\19.00064 11.pdf
../resumes\19.00064 2.pdf
../resumes\19.00064 3.pdf
../resumes\19.00064 4.pdf
../resumes\19.00064 5.pdf
../resumes\19.00064 6.pdf
../resumes\19.00064 7.pdf
../resumes\19.00064 8.pdf
../resumes\19.00064 9.pdf
../resumes\20.00432 0.pdf
../resumes\20.00432 1.pdf
../resumes\20.00432 2.pdf
../resumes\20.00432 3.pdf
../resumes\20.00432 4.pdf
../resumes\20.00973 0.pdf
../resumes\20.00973 1.pdf
../resumes\20.00973 10.pdf
../resumes\20.00973 11.pdf
../resumes\20.00973 12.pdf
../resumes\20.00973 2.pdf
../resumes\20.00973 3.pdf
../resumes\20.00973 4.pdf
../resumes\20.00973 5.pdf
../resumes\20.00973 6.pdf
../resumes\20.00973 7.pdf
../resumes\20.00973 8.pdf
../resumes\20.00973 9.pdf
../resu

In [141]:
resumes = resumes.append(resumesTest)

## Convert text into a vector 

In [152]:
from sklearn.model_selection import train_test_split  
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,2))


# Do not call fit transform twice because it is stored in vectorizer 
X_train = vectorizer.fit_transform(resumes['text'])
X_test = vectorizer.transform(resumesTest['text'])
y_train = resumes['classification']

## Train Classifier

In [153]:
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)  
text_clf  = classifier.fit(X_train, y_train) 

In [159]:
file = '../downloaded/20.02373 CDOT NPS Western Slope_Ulteig.pdf'
fileReader = PyPDF2.PdfFileReader(open(file,'rb'))
count = fileReader.numPages

text = ""
i = 0
X_test = []
while i < count:
    text = fileReader.getPage(i).extractText()
    X_test.append(text)
    
    i = i +1  

#X_test = vectorizer.transform(X_test)
#text_clf.predict(X_test)

In [163]:
X_test[2]

'Statement of Interest'

## Make a prediction

In [150]:
text_clf.predict(X_test), X_test

(array(['NotResume', 'Resume', 'NotResume'], dtype=object),
 <3x14887 sparse matrix of type '<class 'numpy.float64'>'
 	with 16932 stored elements in Compressed Sparse Row format>)